<a href="https://colab.research.google.com/github/selinagalka1/mgs/blob/main/Topic_modelling_with_BERT_sentences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Navigate





In [35]:
from google.colab import drive
drive.mount('/content/drive')

import os

print(os.listdir())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['stopwords_edited.txt', 'schwerin.mm', 'document_info_output.json', 'topic_info_output.json', 'bertopic_results_with_paragraphs.json']


In [36]:
%cd /content/drive/MyDrive/topic_modelling

/content/drive/MyDrive/topic_modelling


In [37]:
!pip install bertopic

# Installing

In [38]:
!pip install pandas

In [39]:
from bertopic import BERTopic
import pandas as pd

# Prepare data

In [40]:
import nltk
from nltk.tokenize import sent_tokenize
import re

nltk.download('punkt')

file_path = './schwerin_text_paragraphs.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

def clean_text(sentence):
    # Replace non-breaking spaces and other non-standard spaces
    sentence = sentence.replace('\xa0', ' ').replace('\n', ' ')
    # Replace multiple spaces with a single space
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    return sentence

def process_text_for_bertopic(text):
    # Split the text into paragraphs using '///' as the delimiter - this was preprocessed with TEI to text with a XSLT stylesheet
    paragraphs = text.split('///')
    sentences = []
    sentence_paragraph_map = []

    # Enumerate the paragraphs and tokenize each into sentences
    for idx, paragraph in enumerate(paragraphs):
        # Trim whitespace which might be left from splitting
        trimmed_paragraph = paragraph.strip()
        if trimmed_paragraph:  # Ensure the paragraph is not empty
            # Tokenize the paragraph into sentences
            paragraph_sentences = sent_tokenize(trimmed_paragraph)
            # Clean and extend the sentences to the main list
            cleaned_sentences = [clean_text(sentence) for sentence in paragraph_sentences]
            sentences.extend(cleaned_sentences)
            # Store the paragraph number for each sentence for later analysis
            sentence_paragraph_map.extend([idx + 1] * len(cleaned_sentences))

    return sentences, sentence_paragraph_map

# Process the text for BERTopic
sentences, sentence_paragraph_map = process_text_for_bertopic(text)


print("The text contains", len(sentences), "based on the tokenization with NLTK.")

# Example: Print each sentence with its paragraph number
for sentence, paragraph_num in zip(sentences, sentence_paragraph_map):
    print(f"Paragraph {paragraph_num}: {sentence}")

# sentences can be directly used for BERTopic modeling


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Paragraph 376: Je n’avais pas un moment de repos pendant trois fois vingt-quatre heures que cela dura.
Paragraph 376: Il me vint plusieurs fois à l’esprit que mon changement de religion m’apportait tant de maux, mais vous me fîtes la grâce, ô mon Dieu, de m’apercevoir d’abord que c’était des tentations.
Paragraph 376: Cela me fit résoudre de redoubler mon vœu et on m’entendit répéter une infinité de fois : « Je veux pourtant y aller, dussai-je m’y faire porter.
Paragraph 376: Ô mon Dieu, assistez-moi.
Paragraph 376: » Je n’en disais pas davantage et je les laissais dans l’erreur qu’ils me croyaient des transports au cerveau.
Paragraph 377: Enfin vers minuit du troisième jour je m’assoupis, mes douleurs me quittèrent tout à fait, et il me prit une sueur extraordinaire que la science du médecin n’avait pu m’attirer malgré tout son art.
Paragraph 377: Je crois que mon sommeil dura bien neuf ou dix heures.
Paragraph 377: Au

In [41]:
print(sentence_paragraph_map)

[1, 2, 3, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 28, 28, 28

# Create topics

In [42]:
topic_model = BERTopic(verbose=True, language="french")

In [43]:
# topics, probs = topic_model.fit_transform(dftexts)
topics, probs = topic_model.fit_transform(sentences)

2024-09-20 10:55:14,321 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/326 [00:00<?, ?it/s]

2024-09-20 11:02:56,589 - BERTopic - Embedding - Completed ✓
2024-09-20 11:02:56,591 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-09-20 11:03:11,515 - BERTopic - Dimensionality - Completed ✓
2024-09-20 11:03:11,517 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-09-20 11:03:12,122 - BERTopic - Cluster - Completed ✓
2024-09-20 11:03:12,135 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-09-20 11:03:12,538 - BERTopic - Representation - Completed ✓


# Save the model

In [44]:
topic_model.save("./schwerin.mm")

2024-09-20 11:03:13,078 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [45]:
 topic_model = BERTopic.load("./schwerin.mm")

# Finetuning the model

In [46]:
from sklearn.feature_extraction.text import CountVectorizer

# Define the stopwords
#stop_words = ["et", "in", "est", "ad", "qui", "autem", "non", "de", "eius", "ut", "quae", "cum", "si", "eum"]

with open('./stopwords_edited.txt', 'r') as file:
  stop_words = [word.strip() for word in file.readlines()]

vectorizer_model = CountVectorizer(stop_words=stop_words)
topic_model.update_topics(sentences, vectorizer_model=vectorizer_model)

# Investigate the topics

In [47]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4305,-1_temps_faire_tous_dieu,"[temps, faire, tous, dieu, grand, être, peu, f...","[Dès que le repas fut fini, elle me dit que da..."
1,0,1105,0_lettre_roi_dire_madame,"[lettre, roi, dire, madame, maison, fort, fair...",[Je dis toutes ces nouvelles au comte de Schwe...
2,1,411,1_dame_femme_air_voulut,"[dame, femme, air, voulut, gracieusement, fill...","[Elle le fit., Elle me le dit., », lui dit-elle.]"
3,2,333,2_prince_roi_reine_empereur,"[prince, roi, reine, empereur, impératrice, pr...",[De là je fus chez madame la Princesse Electri...
4,3,210,3_oui_donc_voulez_deviendrai,"[oui, donc, voulez, deviendrai, donnez, hé, co...","[Je lui dis que oui., », dis-je., Je le lui dis.]"
...,...,...,...,...,...
115,114,11,114_embrasser_baiser_embrassa_courus,"[embrasser, baiser, embrassa, courus, serrant,...","[» Je courus l’embrasser., Je voulais embrasse..."
116,115,11,115_souffrais_sentis_odieux_affecté,"[souffrais, sentis, odieux, affecté, stérilité...","[J’avais compagnie, et tout le monde prit mati..."
117,116,11,116_inexprimables_effectivement_levant_tabourets,"[inexprimables, effectivement, levant, taboure...",[Je m’y mis à genoux sur des tabourets et je s...
118,117,11,117_migraine_tête_désordre_terrible,"[migraine, tête, désordre, terrible, terribles...",[Elle me dit qu’une migraine causait le désord...


In [48]:
# Convert the DataFrame to a JSON string
topic_info_json = topic_info.to_json(orient='records', force_ascii=False, indent=4)

# Specify the output file path
output_file = './topic_info_output.json'

# Save the JSON data to a file
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(topic_info_json)

print(f"Document info has been saved to {output_file}")



Document info has been saved to ./topic_info_output.json


In [49]:
topic_model.get_topic(0)

[('lettre', 0.006898030765731881),
 ('roi', 0.006631891504496464),
 ('dire', 0.006423657341341791),
 ('madame', 0.006377349180808611),
 ('maison', 0.006246602187486796),
 ('fort', 0.00612434462988679),
 ('faire', 0.006026910241659566),
 ('jour', 0.005892056573223652),
 ('après', 0.0057640008199558584),
 ('cour', 0.005331420934396058)]

In [50]:
topic_labels = topic_model.generate_topic_labels(nr_words=20, topic_prefix=False, separator=" - ")
topic_model.set_topic_labels(topic_labels)
# topic_model.set_topic_labels({2: "Lineages"})

In [51]:
topic_model.get_document_info(sentences)

,Document,Topic,Name,CustomName,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,"<?xml version=""1.0"" encoding=""UTF-8""?>Fürst Kh...",31,31_nb_aliae_auxilium_aut,nb - aliae - auxilium - aut - hominum - in - p...,"[nb, aliae, auxilium, aut, hominum, in, provid...","[[NB.], [NB., [NB de son ?]",nb - aliae - auxilium - aut - hominum - in - p...,1.000000,False
1,« Histoire De la Vie de madame la comtesse de ...,0,0_lettre_roi_dire_madame,lettre - roi - dire - madame - maison - fort -...,"[lettre, roi, dire, madame, maison, fort, fair...",[Je dis toutes ces nouvelles au comte de Schwe...,lettre - roi - dire - madame - maison - fort -...,0.951648,False
2,écrite par elle-même à ses enfants suivant les...,80,80_pologne_envoyé_françaises_avantageusement,pologne - envoyé - françaises - avantageusemen...,"[pologne, envoyé, françaises, avantageusement,...",[Le comte de Wilcke revint aussi dans ce temps...,pologne - envoyé - françaises - avantageusemen...,0.725652,False
3,Première partie,42,42_partie____________________________troisième...,partie - __________________________ - troisièm...,"[partie, __________________________, troisième...","[Quatrième partie, Première partie, Troisième ...",partie - __________________________ - troisièm...,0.435091,True
4,L’histoire de ma vie est si remarquable et si ...,-1,-1_temps_faire_tous_dieu,temps - faire - tous - dieu - grand - être - p...,"[temps, faire, tous, dieu, grand, être, peu, f...","[Dès que le repas fut fini, elle me dit que da...",temps - faire - tous - dieu - grand - être - p...,0.000000,False
...,...,...,...,...,...,...,...,...,...
10406,"Cela me rendit si hardie dans la suite, que la...",-1,-1_temps_faire_tous_dieu,temps - faire - tous - dieu - grand - être - p...,"[temps, faire, tous, dieu, grand, être, peu, f...","[Dès que le repas fut fini, elle me dit que da...",temps - faire - tous - dieu - grand - être - p...,0.000000,False
10407,Je crois qu’avec cette période je pourrai fini...,-1,-1_temps_faire_tous_dieu,temps - faire - tous - dieu - grand - être - p...,"[temps, faire, tous, dieu, grand, être, peu, f...","[Dès que le repas fut fini, elle me dit que da...",temps - faire - tous - dieu - grand - être - p...,0.000000,False
10408,"A Dieu seul le Père, le Fils, et Saint-Esprit ...",-1,-1_temps_faire_tous_dieu,temps - faire - tous - dieu - grand - être - p...,"[temps, faire, tous, dieu, grand, être, peu, f...","[Dès que le repas fut fini, elle me dit que da...",temps - faire - tous - dieu - grand - être - p...,0.000000,False
10409,/ Amen.,-1,-1_temps_faire_tous_dieu,temps - faire - tous - dieu - grand - être - p...,"[temps, faire, tous, dieu, grand, être, peu, f...","[Dès que le repas fut fini, elle me dit que da...",temps - faire - tous - dieu - grand - être - p...,0.000000,False


In [52]:
# Convert the DataFrame to a JSON string
document_info_json = document_info.to_json(orient='records', force_ascii=False, indent=4)

# Specify the output file path
output_file = './document_info_output.json'

# Save the JSON data to a file
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(document_info_json)

print(f"Document info has been saved to {output_file}")


Document info has been saved to ./document_info_output.json


# Visualize

In [53]:
topic_model.visualize_documents(sentences)

In [54]:
topic_model.visualize_heatmap()

In [55]:
topic_model.visualize_barchart(width=280, height=330, top_n_topics=40, n_words=10)

# Map documents to paragraphs and generate mapped Output JSON

In [56]:
import json

# File paths
bertopic_output_file = './document_info_output.json'
output_file = './bertopic_results_with_paragraphs.json'

# Load BERTopic output from the JSON file
with open(bertopic_output_file, 'r', encoding='utf-8') as f:
    bertopic_results = json.load(f)


# Ensure that we are mapping the correct sentences from BERTopic to their paragraph numbers
# Assuming that the documents in `bertopic_results` match the sentences exactly

# We will now enhance the BERTopic results by adding paragraph numbers
for idx, result in enumerate(bertopic_results):
    if idx < len(sentence_paragraph_map):
        # Add the corresponding paragraph number from the map
        paragraph_num = sentence_paragraph_map[idx]
        result['Paragraph'] = paragraph_num
    else:
        result['Paragraph'] = None  # Just in case there are more results than sentences

# Save the enhanced results with paragraph mapping back to a new JSON file
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(bertopic_results, f, ensure_ascii=False, indent=4)

print(f"BERTopic results with paragraph mapping saved to {output_file}")


BERTopic results with paragraph mapping saved to ./bertopic_results_with_paragraphs.json


# Analyze topics in paragraphs

In [62]:
import json
from collections import defaultdict
from itertools import combinations

# Load the mapped BERTopic output from the JSON file
mapped_output_path = './bertopic_results_with_paragraphs.json'
with open(mapped_output_path, 'r', encoding='utf-8') as file:
    mapped_output = json.load(file)

# Create a dictionary to map paragraphs to topics
paragraph_topics = defaultdict(set)

for doc in mapped_output:
    paragraph_number = doc['Paragraph']
    topic = doc['Topic']
    # Consider only topics 1-110 - does that make sense?
    if 1 <= topic <= 110:
        paragraph_topics[paragraph_number].add(topic)

# Convert the defaultdict to a regular dict for easier printing
paragraph_topics = dict(paragraph_topics)

# Print the paragraph to topics mapping
print("Paragraph to Topics Mapping (which relevant topics appear in each paragraph):")
for paragraph, topics in paragraph_topics.items():
    print(f"Paragraph {paragraph}: Topics {topics}")

# Analyze topic co-occurrence within paragraphs
topic_cooccurrence = defaultdict(int)

for topics in paragraph_topics.values():
    for topic1, topic2 in combinations(topics, 2):
        topic_cooccurrence[frozenset([topic1, topic2])] += 1

# Convert the defaultdict to a regular dict for easier manipulation
topic_cooccurrence = dict(topic_cooccurrence)

# Print topic co-occurrence results
#print("\nTopic Co-occurrence:")
#for topics, count in topic_cooccurrence.items():
    #topic_pair = list(topics)
    #print(f"Topics {topic_pair[0]} and {topic_pair[1]} co-occur in {count} paragraphs")

# Identify the most frequent co-occurring topic pairs
most_frequent_cooccurrences = sorted(topic_cooccurrence.items(), key=lambda x: x[1], reverse=True)

# Define a threshold for relevance (e.g., top 5 most frequent pairs or frequency count threshold)
threshold = 5  # Top 5 most frequent pairs

# Print the most frequent co-occurring topic pairs
# print("\nMost Frequent Co-occurring Topics:")
# for topics, count in most_frequent_cooccurrences[:threshold]:
    #topic_pair = list(topics)
    #print(f"Topics {topic_pair[0]} and {topic_pair[1]} co-occur in {count} paragraphs")

# You can also define a frequency count threshold if preferred
frequency_threshold = 3  # Adjust this value based on your data

#print("\nRelevant Co-occurring Topics (Frequency Threshold):")
#for topics, count in most_frequent_cooccurrences:
    #if count >= frequency_threshold:
        #topic_pair = list(topics)
        #print(f"Topics {topic_pair[0]} and {topic_pair[1]} co-occur in {count} paragraphs")


Paragraph to Topics Mapping (which relevant topics appear in each paragraph):
Paragraph 2: Topics {1}
Paragraph 4: Topics {15}
Paragraph 5: Topics {5, 29}
Paragraph 6: Topics {29, 62, 7}
Paragraph 7: Topics {64, 14}
Paragraph 8: Topics {66}
Paragraph 9: Topics {28}
Paragraph 10: Topics {8}
Paragraph 11: Topics {37}
Paragraph 13: Topics {40, 1, 2, 28}
Paragraph 14: Topics {5, 7, 40, 17, 29}
Paragraph 15: Topics {3, 5, 102, 110, 20, 23, 57}
Paragraph 16: Topics {1, 5, 7, 45, 22}
Paragraph 17: Topics {91, 85}
Paragraph 18: Topics {1, 12}
Paragraph 19: Topics {1, 8, 43, 14, 29}
Paragraph 20: Topics {5, 4, 29}
Paragraph 21: Topics {8, 10, 22}
Paragraph 22: Topics {26, 34, 47}
Paragraph 23: Topics {91, 5, 22, 23}
Paragraph 24: Topics {1, 59}
Paragraph 25: Topics {1, 5, 38, 8, 60}
Paragraph 26: Topics {5, 7, 9, 74, 57}
Paragraph 27: Topics {57, 43, 87, 7}
Paragraph 28: Topics {8, 92, 45}
Paragraph 29: Topics {5, 8, 108, 44, 22}
Paragraph 30: Topics {5, 7, 8, 71, 85, 22, 25, 28}
Paragraph 31: 